In [1]:
import requests
from bs4 import BeautifulSoup
import re

# URL of the Ollama model library
OLLAMA_MODEL_LIBRARY_URL = "https://ollama.com/library"

def fetch_available_models():
    """Scrapes the Ollama model library and correctly pairs models with their parameter sizes."""
    response = requests.get(OLLAMA_MODEL_LIBRARY_URL)
    if response.status_code != 200:
        print("❌ Failed to fetch model list from Ollama.")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    models = []

    # Find all links containing model names (/library/{model_name})
    for link in soup.find_all("a", href=True):
        if link["href"].startswith("/library/"):
            model_name = link["href"].split("/")[-1]  # Extract model name from URL

            # Look for size in the same block (search text within the same section)
            parent = link.find_parent()
            size_text = "Unknown"
            if parent:
                text_in_block = parent.get_text(" ", strip=True)
                size_matches = re.findall(r"(\d+(\.\d+)?b)", text_in_block, re.IGNORECASE)
                sizes = [s[0] for s in size_matches]  # Extract sizes like "7b", "70b", etc.

                # Store the largest size or all available ones
                size_text = ", ".join(sizes) if sizes else "Unknown"

            models.append((model_name, size_text))

    return models

    

In [2]:
print("🔍 Fetching available models from the Ollama model library...")
models = fetch_available_models()

print("\n📌 Available Ollama Models:")
for i, (name, size) in enumerate(models, 1):
    print(f"{i}) {name} - {size}", end=', ')

🔍 Fetching available models from the Ollama model library...

📌 Available Ollama Models:
1) gemma3 - 1b, 4b, 12b, 27b, 2) qwq - 32b, 3) deepseek-r1 - 1.5b, 7b, 8b, 14b, 32b, 70b, 671b, 4) llama3.3 - 70B, 70B, 405B, 70b, 5) phi4 - 14B, 14b, 6) llama3.2 - 1B, 3B, 1b, 3b, 7) llama3.1 - 8B, 70B, 405B, 8b, 70b, 405b, 8) nomic-embed-text - Unknown, 9) mistral - 7B, 7b, 10) llama3 - 8b, 70b, 11) qwen2.5 - 0.5b, 1.5b, 3b, 7b, 14b, 32b, 72b, 12) qwen2.5-coder - 0.5b, 1.5b, 3b, 7b, 14b, 32b, 13) llava - 7b, 13b, 34b, 14) qwen - 0.5B, 110B, 0.5b, 1.8b, 4b, 7b, 14b, 32b, 72b, 110b, 15) gemma - 2b, 7b, 16) qwen2 - 0.5b, 1.5b, 7b, 72b, 17) gemma2 - 2B, 9B, 27B, 2b, 9b, 27b, 18) llama2 - 7B, 70B, 7b, 13b, 70b, 19) phi3 - 3B, 14B, 3.8b, 14b, 20) mxbai-embed-large - Unknown, 21) codellama - 7b, 13b, 34b, 70b, 22) llama3.2-vision - 11B, 90B, 11b, 90b, 23) mistral-nemo - 12B, 12b, 24) tinyllama - 1.1B, 1.1b, 25) deepseek-v3 - 671B, 37B, 671b, 26) starcoder2 - 3B, 7B, 15B, 3b, 7b, 15b, 27) llama2-uncensor

In [3]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# Generate synthetic data: 100 samples with 10 features each
X = np.random.rand(100, 10)
y = np.random.rand(100, 1)

# Convert numpy arrays to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Create a TensorDataset and then split it into training and test sets
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Diagnostic Output
print(f"Train dataset size: {len(train_dataset)}, Test dataset size: {len(test_dataset)}")

# Show shapes of the first batch
first_train_sample = train_dataset[0]
print(f"First training sample - X shape: {first_train_sample[0].shape}, y shape: {first_train_sample[1].shape}")

# Show content of the first training sample
print(f"First training sample - X: {first_train_sample[0].numpy()}, y: {first_train_sample[1].numpy()}")

Train dataset size: 80, Test dataset size: 20
First training sample - X shape: torch.Size([10]), y shape: torch.Size([1])
First training sample - X: [0.82811207 0.73099226 0.6460811  0.98673284 0.5317019  0.11775014
 0.99185634 0.32278016 0.5298668  0.3955642 ], y: [0.8041298]


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Instantiate the model
input_size = 10
hidden_size = 16
output_size = 1
model = SimpleNN(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print(model)

SimpleNN(
  (fc1): Linear(in_features=10, out_features=16, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=16, out_features=1, bias=True)
)


In [5]:
from torch.utils.data import DataLoader

# Create a DataLoader for the training dataset
dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Example: Iterate through one batch
n = 1
for batch_X, batch_y in dataloader:
    print(f"{n}) Batch X shape:", batch_X.size())
    print("      Batch y shape:", batch_y.size())
    n += 1


1) Batch X shape: torch.Size([16, 10])
      Batch y shape: torch.Size([16, 1])
2) Batch X shape: torch.Size([16, 10])
      Batch y shape: torch.Size([16, 1])
3) Batch X shape: torch.Size([16, 10])
      Batch y shape: torch.Size([16, 1])
4) Batch X shape: torch.Size([16, 10])
      Batch y shape: torch.Size([16, 1])
5) Batch X shape: torch.Size([16, 10])
      Batch y shape: torch.Size([16, 1])


In [6]:
16*5

80